# Topic Modeling

* Memodelkan Topik pada Dataset Spam Tipe-3 dan Hasil StructuredBased
* Pada Code ini hanya menggunakan model LSA yang mana merupakan pemodelan terbaik dari variasi skenario pengujian 

In [ ]:
# Works based on https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24

import pandas as pd
from gensim.models.lsimodel import LsiModel
import matplotlib.pyplot as plt
import gensim
from gensim.models import CoherenceModel
from gensim.models import Phrases
import pyLDAvis
from random import randint
import ast
import logging

# Coherence Score

* Menghitung Coherence Score untuk Menentukan Jumlah Topik Terbaik

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=3, step=1):
        """
        Compute c_v coherence for various number of topics
    
        Parameters:
        ----------
        dictionary : Gensim dictionary
        corpus : Gensim corpus
        texts : List of input texts
        limit : Max num of topics
    
        Returns:
        -------
        model_list : List of  topic models
        coherence_values : Coherence values corresponding to the  model with respective number of topics
        """
        coherence_values = []
        model_list = []
        print("start")
        for num_topics in range(start, limit, step):
            print("num topic:")
            model = LsiModel(corpus, num_topics=num_topics, id2word=dictionary, chunksize=2000)
            print(num_topics)
            model_list.append(model)
            coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
            coherence_values.append(coherencemodel.get_coherence()) 
        return model_list, coherence_values

# Data 

* Input : Data Hasil dari StructuredBased.ipynb
* Output : Model Topik 

In [ ]:
def main ():

    '''
    def tokenize(text):
        from nltk.tokenize import word_tokenize
        tokens = word_tokenize(str(text))
        return tokens
    for i in range(len(data)):
        clean.append(tokenize(data['clean'][i]))
    ''' 
    #Data Hasil dari StructuredBased.ipynb
    data = pd.read_csv('Data/Output/1. Preprocessing/amz_spam323_token_ag.csv', sep=',', encoding='latin-1')
    clean=data['clean']
    clean = list(map(ast.literal_eval,clean))
    
    #Membuat token bigram 
    bigram = Phrases(clean, min_count=10)
    for idx in range(len(clean)):
        for token in bigram[clean[idx]]:
            if '_' in token:
                clean[idx].append(token)
    
    #Membuat kamus kata
    
    dictionary = gensim.corpora.Dictionary(clean)
    
    #Preview
    count = 0
    for k, v in dictionary.iteritems():
        print(k, v)
        count += 1
        if count > 5:
            break
    
    dictionary.filter_extremes()
    
    #Membuat bag of word (kemunculan kata)
    bow_corpus = [dictionary.doc2bow(doc) for doc in clean]
    print("bow")
    print("coherence")
    print("done")
    model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=bow_corpus, texts=clean, start=3, limit=8, step=1)
    
    # Graf Model topik dan Coh_score
    limit=8; start=3; step=1;
    x = range(start, limit, step)
    plt.plot(x, coherence_values)
    plt.xlabel("Num Topics on BoW corpus")
    plt.ylabel("Coherence score")
    plt.legend(("coherence_values"))
    plt.show()
    
    print(coherence_values)
    lsimodel_model = model_list   
    
    return lsimodel_model, coherence_values

# Save Model Topik Terbaik

* berdasarkan Coherence Score Tertinggi

In [ ]:
if __name__ == "__main__":
    lsimodel_model, coherence_values=main()
    print(lsimodel_model)
    print(coherence_values)
    for idx, topic in lsimodel_model[2].print_topics(5): 
        print('Topic: {} \nWords: {}'.format(idx, topic))
        
    #lsimodel_model[4].save('Data/Output/2. Topic Modeling/lsimodelmodel_26_12_3 topik_best2') #array lsimodel_model disesuaikan dengan coh_score tertinggi
    #model terbaik : lsimodelmodel_26_12_3 topik_best2